In [12]:
# ! pip install pandas-datareader
# ! pip install yfinance

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.2/63.2 kB 3.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 112.2/112.2 kB 6.0 MB/s eta 0:00:00


In [7]:
import bs4 as bs
import datetime as dt
import os
import pandas as pd
import pandas_datareader.data as web
import pickle
import requests 
import yfinance as yfin
import numpy as np

yfin.pdr_override()


def save_sp500_tickers():
    resp = requests.get('http://en.wikipedia.org/wiki/List_of_S%26P_500_companies')
    soup = bs.BeautifulSoup(resp.text, 'lxml')
    table = soup.find('table', {'class': 'wikitable sortable'})
    tickers = []
    for row in table.findAll('tr')[1:]:
        ticker = row.findAll('td')[0].text
        tickers.append(ticker)

    with open("sp500tickers.pickle","wb") as f:
        pickle.dump(tickers,f)

    return tickers
# save_sp500_tickers()

def get_data_from_yahoo(reload_sp500=False):
    if reload_sp500:
        tickers = save_sp500_tickers()
    else:
        with open("sp500tickers.pickle", "rb") as f:
            tickers = pickle.load(f)
    if not os.path.exists('stock_dfs'):
        os.makedirs('stock_dfs')

    start = dt.datetime(2000, 1, 1)
    end = dt.datetime.now()
    print(start, end)
    for ticker in tickers:
        print(ticker)
        # just in case your connection breaks, we'd like to save our progress!
        if not os.path.exists('stock_dfs/{}.csv'.format(ticker)):
            df = web.get_data_yahoo(ticker, start, end)
            df.reset_index(inplace=True)
            df.set_index("Date", inplace=True)
            df['Symbol'] = ticker
            df.to_csv('stock_dfs/{}.csv'.format(ticker))
        else:
            print('Already have {}'.format(ticker))

def compile_data():
    with open("sp500tickers.pickle", "rb") as f:
        tickers = pickle.load(f)

    main_df = pd.DataFrame()

    for count, ticker in enumerate(tickers):
        df = pd.read_csv('stock_dfs/{}.csv'.format(ticker))
        df.set_index('Date', inplace=True)
        ticker = ticker.replace('\n', '')
        # df.rename(columns={'Adj Close': ticker}, inplace=True)
        df.drop(['Open', 'High', 'Low', 'Close', 'Volume', 'Symbol'], axis=1, inplace=True)
        # df['return_pct'] = df['Adj Close'].pct_change()
        # df['return_pct'] = df['return_pct'].fillna(0)
        df[ticker] = np.log(df['Adj Close']/df['Adj Close'].shift(1))
        df.drop(['Adj Close'], axis=1, inplace=True)
        df = df.dropna()
        # df['return_log'] = df['return_log'].fillna(0)
        
        # df.rename(columns={'Adj Close': ticker}, inplace=True)
        if main_df.empty:
            main_df = df
        else:
            main_df = main_df.join(df, how='outer')

        # if count % 10 == 0:
        #     print(count)
    print(main_df.head())
    main_df.to_csv('sp500_joined_closes.csv')

In [8]:

get_data_from_yahoo(True)


2000-01-01 00:00:00 2023-07-04 22:52:22.050606
MMM

[*********************100%***********************]  1 of 1 completed
AOS

[*********************100%***********************]  1 of 1 completed
ABT

[*********************100%***********************]  1 of 1 completed
ABBV

[*********************100%***********************]  1 of 1 completed
ACN

[*********************100%***********************]  1 of 1 completed
ATVI

[*********************100%***********************]  1 of 1 completed
ADM

[*********************100%***********************]  1 of 1 completed
ADBE

[*********************100%***********************]  1 of 1 completed
ADP

[*********************100%***********************]  1 of 1 completed
AAP

[*********************100%***********************]  1 of 1 completed
AES

[*********************100%***********************]  1 of 1 completed
AFL

[*********************100%***********************]  1 of 1 completed
A

[*********************100%***********************]  1 of 1 


1 Failed download:
['BRK.B']: Exception('%ticker%: No timezone found, symbol may be delisted')



BBY

[*********************100%***********************]  1 of 1 completed
BIO

[*********************100%***********************]  1 of 1 completed
TECH

[*********************100%***********************]  1 of 1 completed
BIIB

[*********************100%***********************]  1 of 1 completed
BLK

[*********************100%***********************]  1 of 1 completed
BK

[*********************100%***********************]  1 of 1 completed
BA

[*********************100%***********************]  1 of 1 completed
BKNG

[*********************100%***********************]  1 of 1 completed
BWA

[*********************100%***********************]  1 of 1 completed
BXP

[*********************100%***********************]  1 of 1 completed
BSX

[*********************100%***********************]  1 of 1 completed
BMY

[*********************100%***********************]  1 of 1 completed
AVGO

[*********************100%***********************]  1 of 1 completed
BR

[*********************100%*****


1 Failed download:
['BF.B']: Exception('%ticker%: No price data found, symbol may be delisted (1d 2000-01-01 00:00:00 -> 2023-07-04 22:52:22.050606)')



BG

[*********************100%***********************]  1 of 1 completed
CHRW

[*********************100%***********************]  1 of 1 completed
CDNS

[*********************100%***********************]  1 of 1 completed
CZR

[*********************100%***********************]  1 of 1 completed
CPT

[*********************100%***********************]  1 of 1 completed
CPB

[*********************100%***********************]  1 of 1 completed
COF

[*********************100%***********************]  1 of 1 completed
CAH

[*********************100%***********************]  1 of 1 completed
KMX

[*********************100%***********************]  1 of 1 completed
CCL

[*********************100%***********************]  1 of 1 completed
CARR

[*********************100%***********************]  1 of 1 completed
CTLT

[*********************100%***********************]  1 of 1 completed
CAT

[*********************100%***********************]  1 of 1 completed
CBOE

[*********************100%**

In [9]:
save_sp500_tickers()

['MMM\n',
 'AOS\n',
 'ABT\n',
 'ABBV\n',
 'ACN\n',
 'ATVI\n',
 'ADM\n',
 'ADBE\n',
 'ADP\n',
 'AAP\n',
 'AES\n',
 'AFL\n',
 'A\n',
 'APD\n',
 'AKAM\n',
 'ALK\n',
 'ALB\n',
 'ARE\n',
 'ALGN\n',
 'ALLE\n',
 'LNT\n',
 'ALL\n',
 'GOOGL\n',
 'GOOG\n',
 'MO\n',
 'AMZN\n',
 'AMCR\n',
 'AMD\n',
 'AEE\n',
 'AAL\n',
 'AEP\n',
 'AXP\n',
 'AIG\n',
 'AMT\n',
 'AWK\n',
 'AMP\n',
 'ABC\n',
 'AME\n',
 'AMGN\n',
 'APH\n',
 'ADI\n',
 'ANSS\n',
 'AON\n',
 'APA\n',
 'AAPL\n',
 'AMAT\n',
 'APTV\n',
 'ACGL\n',
 'ANET\n',
 'AJG\n',
 'AIZ\n',
 'T\n',
 'ATO\n',
 'ADSK\n',
 'AZO\n',
 'AVB\n',
 'AVY\n',
 'AXON\n',
 'BKR\n',
 'BALL\n',
 'BAC\n',
 'BBWI\n',
 'BAX\n',
 'BDX\n',
 'WRB\n',
 'BRK.B\n',
 'BBY\n',
 'BIO\n',
 'TECH\n',
 'BIIB\n',
 'BLK\n',
 'BK\n',
 'BA\n',
 'BKNG\n',
 'BWA\n',
 'BXP\n',
 'BSX\n',
 'BMY\n',
 'AVGO\n',
 'BR\n',
 'BRO\n',
 'BF.B\n',
 'BG\n',
 'CHRW\n',
 'CDNS\n',
 'CZR\n',
 'CPT\n',
 'CPB\n',
 'COF\n',
 'CAH\n',
 'KMX\n',
 'CCL\n',
 'CARR\n',
 'CTLT\n',
 'CAT\n',
 'CBOE\n',
 'CBRE\n',
 'CD

In [10]:
compile_data()


                 MMM       AOS       ABT  ABBV  ACN      ATVI       ADM   
Date                                                                      
2000-01-04 -0.040546 -0.014514 -0.028988   NaN  NaN -0.030891 -0.010472  \
2000-01-05  0.028554 -0.002929 -0.001839   NaN  NaN  0.003913 -0.015915   
2000-01-06  0.077358 -0.014771  0.034393   NaN  NaN -0.019724  0.005333   
2000-01-07  0.019657  0.040822  0.010619   NaN  NaN  0.027506  0.015831   
2000-01-10 -0.004879  0.011363 -0.007067   NaN  NaN  0.049159  0.000000   

                ADBE       ADP  AAP  ...  WTW       GWW  WYNN       XEL  XYL   
Date                                 ...                                       
2000-01-04 -0.087618  0.000000  NaN  ...  NaN -0.028987   NaN  0.022765  NaN  \
2000-01-05  0.019578 -0.009661  NaN  ...  NaN  0.013909   NaN  0.037860  NaN   
2000-01-06  0.008130  0.013260  NaN  ...  NaN -0.004153   NaN -0.009331  NaN   
2000-01-07  0.047440  0.022500  NaN  ...  NaN -0.015374   NaN  0.000000  N

In [11]:
df = pd.read_csv('sp500_joined_closes.csv')
df.head()

,Date,MMM,AOS,ABT,ABBV,ACN,ATVI,ADM,ADBE,ADP,...,WTW,GWW,WYNN,XEL,XYL,YUM,ZBRA,ZBH,ZION,ZTS
0,2000-01-04,-0.040546,-0.014514,-0.028988,NaN,NaN,-0.030891,-0.010472,-0.087618,0.000000,...,NaN,-0.028987,NaN,0.022765,NaN,-0.020305,-0.014533,NaN,-0.049635,NaN
1,2000-01-05,0.028554,-0.002929,-0.001839,NaN,NaN,0.003913,-0.015915,0.019578,-0.009661,...,NaN,0.013909,NaN,0.037860,NaN,0.005115,0.018963,NaN,-0.001184,NaN
2,2000-01-06,0.077358,-0.014771,0.034393,NaN,NaN,-0.019724,0.005333,0.008130,0.013260,...,NaN,-0.004153,NaN,-0.009331,NaN,-0.008539,-0.055665,NaN,0.014118,NaN
3,2000-01-07,0.019657,0.040822,0.010619,NaN,NaN,0.027506,0.015831,0.047440,0.022500,...,NaN,-0.015374,NaN,0.000000,NaN,-0.022551,-0.011160,NaN,0.002333,NaN
4,2000-01-10,-0.004879,0.011363,-0.007067,NaN,NaN,0.049159,0.000000,0.037883,0.024292,...,NaN,0.042736,NaN,0.000000,NaN,0.039558,0.033114,NaN,-0.018230,NaN


In [12]:
df.tail()

,Date,MMM,AOS,ABT,ABBV,ACN,ATVI,ADM,ADBE,ADP,...,WTW,GWW,WYNN,XEL,XYL,YUM,ZBRA,ZBH,ZION,ZTS
5906,2023-06-27,-0.021645,0.018434,-0.006657,-0.017567,0.011367,0.008715,0.003389,0.020150,0.000325,...,0.005593,0.029516,0.004834,-0.006704,0.008458,0.000372,0.034901,0.013907,0.008091,0.018510
5907,2023-06-28,0.003150,-0.001939,-0.001671,-0.000679,0.003021,-0.006320,0.000676,-0.014079,0.000000,...,-0.001632,0.003226,-0.014381,-0.019405,-0.009633,0.006532,0.002402,-0.005609,-0.007722,-0.009242
5908,2023-06-29,0.006774,0.003460,0.000465,0.005419,0.021671,-0.006239,0.009155,0.002774,0.003935,...,0.002928,0.011972,0.002395,0.000816,0.007211,0.015418,0.018142,0.007954,0.006989,0.016248
5909,2023-06-30,0.008529,0.005511,0.012553,0.011196,0.000810,0.014578,0.012518,0.010732,0.015267,...,0.012606,0.008008,0.010566,0.014256,0.011431,0.009354,0.024466,0.003096,-0.015516,0.001976
5910,2023-07-03,0.010436,-0.005649,-0.016649,0.002076,0.007715,-0.011453,0.015366,-0.007760,-0.004834,...,-0.008615,-0.007676,0.009518,0.012945,-0.007039,-0.009281,0.001486,-0.021451,0.046554,-0.011564
